# Korean

In [14]:
! pip install numpy
! pip install kiwipiepy
! pip install nltk
! pip install datasets
! pip install stanza

### Load Dataset

In [15]:
from datasets import load_dataset
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize  # tokenization

nltk.download('punkt')       # Tokenizer model
nltk.download('punkt_tab')

nltk.download('stopwords')   # List of common stopwords for text normalization
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /Users/park/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /Users/park/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/park/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
dataset_ko = load_dataset("wikimedia/wikipedia", "20231101.ko")

In [17]:
df_ko = pd.DataFrame(dataset_ko["train"])
df_sample_ko = df_ko.sample(n=1000, random_state=42)
df_sample_ko.head()

,id,url,title,text
623706,3582873,https://ko.wikipedia.org/wiki/%EC%B9%B4%ED%83%...,카타쿠라 카네타로,카타쿠라 카네타로 ()는 현재의 카타쿠라 공업의 대대로 이어진 이름이다\n\n 카타...
23304,70654,https://ko.wikipedia.org/wiki/%ED%83%84%EB%91%90,탄두,"탄두(彈頭, )란 미사일, 로켓, 어뢰 등의 추진에 의해 전달되는 폭발성 물질 또는..."
79240,268688,https://ko.wikipedia.org/wiki/%EB%8B%88%EC%8A%...,니스 (프랑스),"니스(, 니사르드 , , , , )는 프랑스 남부의 항만 도시로 프랑스의 지중해 연..."
140214,529155,https://ko.wikipedia.org/wiki/%EA%B9%80%ED%99%...,김홍수 게이트,김홍수 게이트는 90년대 초 초등학교 동창인 법조인 친구를 통해 서울지방법원에서 근...
28599,89713,https://ko.wikipedia.org/wiki/%EC%9D%B8%EB%8F%...,인도아리아어군의 언어 목록,다음은 인도아리아어군의 언어 목록이다.\n\n\n 고전어\n 베다 산스크리트어\n ...


### Removing punctuations and symbols

In [18]:
def clean_text_ko(text):
    text = re.sub(r'\n', ' ', text)  # Remove newline characters
    text = re.sub(r"[^가-힣0-9\s]", "", text) # Keep only Korean chars, digits, and spaces
    text = re.sub(r"[ㄱ-ㅎㅏ-ㅣ]", "", text) # Remove isolated Characters
    text = re.sub(r'\s+', ' ', text).strip()  # multiple spaces
    return text

df_sample_ko["clean_text"] = df_sample_ko["text"].apply(clean_text_ko)

### Tokenization

In [19]:
from kiwipiepy import Kiwi

# Kiwi.analyze returns morphemes and POS tags
kiwi = Kiwi()

def tokenize(text):
    # top_n=1: use the most probable analysis (faster)
    analyzed = kiwi.analyze(text, top_n=1)
    # analyzed[0][0]: list of (morph, POS, start, length)
    tokens = [m[0] for m in analyzed[0][0]]
    return tokens

df_sample_ko["tokens"] = df_sample_ko["clean_text"].apply(tokenize)

In [20]:
df_sample_ko.head()

,id,url,title,text,clean_text,tokens
623706,3582873,https://ko.wikipedia.org/wiki/%EC%B9%B4%ED%83%...,카타쿠라 카네타로,카타쿠라 카네타로 ()는 현재의 카타쿠라 공업의 대대로 이어진 이름이다\n\n 카타...,카타쿠라 카네타로 는 현재의 카타쿠라 공업의 대대로 이어진 이름이다 카타쿠라 카네타...,"[카타쿠라 카네타로, 는, 현재, 의, 카타쿠라, 공업, 의, 대대, 로, 이어지,..."
23304,70654,https://ko.wikipedia.org/wiki/%ED%83%84%EB%91%90,탄두,"탄두(彈頭, )란 미사일, 로켓, 어뢰 등의 추진에 의해 전달되는 폭발성 물질 또는...",탄두 란 미사일 로켓 어뢰 등의 추진에 의해 전달되는 폭발성 물질 또는 독성 물질이...,"[탄두, 이, 란, 미사일, 로켓, 어뢰, 등, 의, 추진, 에, 의하, 어, 전달..."
79240,268688,https://ko.wikipedia.org/wiki/%EB%8B%88%EC%8A%...,니스 (프랑스),"니스(, 니사르드 , , , , )는 프랑스 남부의 항만 도시로 프랑스의 지중해 연...",니스 니사르드 는 프랑스 남부의 항만 도시로 프랑스의 지중해 연안에 위치해 있다 마...,"[니스, 니사르드, 는, 프랑스, 남부, 의, 항만, 도시, 로, 프랑스, 의, 지..."
140214,529155,https://ko.wikipedia.org/wiki/%EA%B9%80%ED%99%...,김홍수 게이트,김홍수 게이트는 90년대 초 초등학교 동창인 법조인 친구를 통해 서울지방법원에서 근...,김홍수 게이트는 90년대 초 초등학교 동창인 법조인 친구를 통해 서울지방법원에서 근...,"[김홍, 수, 게이트, 는, 90, 년대, 초, 초등, 학교, 동창, 이, ᆫ, 법..."
28599,89713,https://ko.wikipedia.org/wiki/%EC%9D%B8%EB%8F%...,인도아리아어군의 언어 목록,다음은 인도아리아어군의 언어 목록이다.\n\n\n 고전어\n 베다 산스크리트어\n ...,다음은 인도아리아어군의 언어 목록이다 고전어 베다 산스크리트어 산스크리트어 미탄니어...,"[다음, 은, 인도아리아어군, 의, 언어, 목록, 이, 다, 고, 전어, 베다, 산..."


### Lemmatization

In [21]:
def lemmatize_korean(tokens):
    lemmas = []
    remove_tags = {"Josa", "Eomi", "Suffix", "Punctuation"}

    for token in tokens:
        result = kiwi.analyze(token, top_n=1)
        if not result:
            continue

        for form, tag, *_ in result[0][0]:
            if tag not in remove_tags:
                lemmas.append(form)
                break
    return lemmas

df_sample_ko["tokens_lemmatized"] = df_sample_ko["tokens"].apply(lemmatize_korean)


In [22]:
df_sample_ko.head()

,id,url,title,text,clean_text,tokens,tokens_lemmatized
623706,3582873,https://ko.wikipedia.org/wiki/%EC%B9%B4%ED%83%...,카타쿠라 카네타로,카타쿠라 카네타로 ()는 현재의 카타쿠라 공업의 대대로 이어진 이름이다\n\n 카타...,카타쿠라 카네타로 는 현재의 카타쿠라 공업의 대대로 이어진 이름이다 카타쿠라 카네타...,"[카타쿠라 카네타로, 는, 현재, 의, 카타쿠라, 공업, 의, 대대, 로, 이어지,...","[카타쿠라 카네타로, 늘, 현재, 의, 카타쿠, 공업, 의, 대대, 로, 이어, ᆫ..."
23304,70654,https://ko.wikipedia.org/wiki/%ED%83%84%EB%91%90,탄두,"탄두(彈頭, )란 미사일, 로켓, 어뢰 등의 추진에 의해 전달되는 폭발성 물질 또는...",탄두 란 미사일 로켓 어뢰 등의 추진에 의해 전달되는 폭발성 물질 또는 독성 물질이...,"[탄두, 이, 란, 미사일, 로켓, 어뢰, 등, 의, 추진, 에, 의하, 어, 전달...","[탄두, 이, 란, 미사일, 로켓, 어뢰, 등, 의, 추진, 에, 의하, 어, 전달..."
79240,268688,https://ko.wikipedia.org/wiki/%EB%8B%88%EC%8A%...,니스 (프랑스),"니스(, 니사르드 , , , , )는 프랑스 남부의 항만 도시로 프랑스의 지중해 연...",니스 니사르드 는 프랑스 남부의 항만 도시로 프랑스의 지중해 연안에 위치해 있다 마...,"[니스, 니사르드, 는, 프랑스, 남부, 의, 항만, 도시, 로, 프랑스, 의, 지...","[니스, 니사르드, 늘, 프랑스, 남부, 의, 항만, 도시, 로, 프랑스, 의, 지..."
140214,529155,https://ko.wikipedia.org/wiki/%EA%B9%80%ED%99%...,김홍수 게이트,김홍수 게이트는 90년대 초 초등학교 동창인 법조인 친구를 통해 서울지방법원에서 근...,김홍수 게이트는 90년대 초 초등학교 동창인 법조인 친구를 통해 서울지방법원에서 근...,"[김홍, 수, 게이트, 는, 90, 년대, 초, 초등, 학교, 동창, 이, ᆫ, 법...","[김홍, 수, 게이트, 늘, 90, 년대, 초, 초등, 학교, 동창, 이, ᆫ, 법..."
28599,89713,https://ko.wikipedia.org/wiki/%EC%9D%B8%EB%8F%...,인도아리아어군의 언어 목록,다음은 인도아리아어군의 언어 목록이다.\n\n\n 고전어\n 베다 산스크리트어\n ...,다음은 인도아리아어군의 언어 목록이다 고전어 베다 산스크리트어 산스크리트어 미탄니어...,"[다음, 은, 인도아리아어군, 의, 언어, 목록, 이, 다, 고, 전어, 베다, 산...","[다음, 은, 인도아리아어군, 의, 언어, 목록, 이, 다, 고, 전어, 베, 산스..."


### Normalization

In [23]:
def load_stopwords(language: str):
    filename = f"stopwords/stopwords-{language}.txt"
    try:
        with open(filename, "r", encoding="utf-8") as f:
            stopwords = set(line.strip() for line in f if line.strip())
        return stopwords
    except FileNotFoundError:
        print(f"'{language}' - language not supported")
        return set()
    
stopwords_korean = load_stopwords("ko")

In [24]:
# Token Normalization Function (Stopword Removal)
stopwords_dict = {
    "ko": set(stopwords_korean),
    # "pt": set(stopwords_portuguese),
    # "it": set(stopwords_italian),
    # "en": set(stopwords_english),
}

def normalize_tokens(tokens, lang):
    if lang in stopwords_dict:
        stops = stopwords_dict[lang]
    else:
        from nltk.corpus import stopwords
        try:
            stops = set(stopwords.words(lang))  # NLTK stopwords for other languages
        except:
            stops = set()  # Fallback if stopwords are unavailable
    # Remove stopwords and single-character tokens
    normalized_tokens = [w for w in tokens if w not in stops and len(w) > 1]

    return normalized_tokens

# Apply Stopword Removal to the Pre-Tokenized Dataset
df_sample_ko["tokens_norm"] = df_sample_ko["tokens_lemmatized"].apply(lambda x: normalize_tokens(x, lang='ko'))

df_sample_ko.head()

,id,url,title,text,clean_text,tokens,tokens_lemmatized,tokens_norm
623706,3582873,https://ko.wikipedia.org/wiki/%EC%B9%B4%ED%83%...,카타쿠라 카네타로,카타쿠라 카네타로 ()는 현재의 카타쿠라 공업의 대대로 이어진 이름이다\n\n 카타...,카타쿠라 카네타로 는 현재의 카타쿠라 공업의 대대로 이어진 이름이다 카타쿠라 카네타...,"[카타쿠라 카네타로, 는, 현재, 의, 카타쿠라, 공업, 의, 대대, 로, 이어지,...","[카타쿠라 카네타로, 늘, 현재, 의, 카타쿠, 공업, 의, 대대, 로, 이어, ᆫ...","[카타쿠라 카네타로, 현재, 카타쿠, 공업, 대대, 이어, 이름, 카타쿠라 카네타로..."
23304,70654,https://ko.wikipedia.org/wiki/%ED%83%84%EB%91%90,탄두,"탄두(彈頭, )란 미사일, 로켓, 어뢰 등의 추진에 의해 전달되는 폭발성 물질 또는...",탄두 란 미사일 로켓 어뢰 등의 추진에 의해 전달되는 폭발성 물질 또는 독성 물질이...,"[탄두, 이, 란, 미사일, 로켓, 어뢰, 등, 의, 추진, 에, 의하, 어, 전달...","[탄두, 이, 란, 미사일, 로켓, 어뢰, 등, 의, 추진, 에, 의하, 어, 전달...","[탄두, 미사일, 로켓, 어뢰, 추진, 의하, 전달, 폭발, 물질, 또는, 물질, ..."
79240,268688,https://ko.wikipedia.org/wiki/%EB%8B%88%EC%8A%...,니스 (프랑스),"니스(, 니사르드 , , , , )는 프랑스 남부의 항만 도시로 프랑스의 지중해 연...",니스 니사르드 는 프랑스 남부의 항만 도시로 프랑스의 지중해 연안에 위치해 있다 마...,"[니스, 니사르드, 는, 프랑스, 남부, 의, 항만, 도시, 로, 프랑스, 의, 지...","[니스, 니사르드, 늘, 프랑스, 남부, 의, 항만, 도시, 로, 프랑스, 의, 지...","[니스, 니사르드, 프랑스, 남부, 항만, 도시, 프랑스, 지중해, 연안, 위치, ..."
140214,529155,https://ko.wikipedia.org/wiki/%EA%B9%80%ED%99%...,김홍수 게이트,김홍수 게이트는 90년대 초 초등학교 동창인 법조인 친구를 통해 서울지방법원에서 근...,김홍수 게이트는 90년대 초 초등학교 동창인 법조인 친구를 통해 서울지방법원에서 근...,"[김홍, 수, 게이트, 는, 90, 년대, 초, 초등, 학교, 동창, 이, ᆫ, 법...","[김홍, 수, 게이트, 늘, 90, 년대, 초, 초등, 학교, 동창, 이, ᆫ, 법...","[김홍, 게이트, 90, 년대, 초등, 학교, 동창, 법조인, 친구, 통하, 서울,..."
28599,89713,https://ko.wikipedia.org/wiki/%EC%9D%B8%EB%8F%...,인도아리아어군의 언어 목록,다음은 인도아리아어군의 언어 목록이다.\n\n\n 고전어\n 베다 산스크리트어\n ...,다음은 인도아리아어군의 언어 목록이다 고전어 베다 산스크리트어 산스크리트어 미탄니어...,"[다음, 은, 인도아리아어군, 의, 언어, 목록, 이, 다, 고, 전어, 베다, 산...","[다음, 은, 인도아리아어군, 의, 언어, 목록, 이, 다, 고, 전어, 베, 산스...","[인도아리아어군, 언어, 목록, 전어, 산스크리트어, 산스크리트어, 미탄니, 프라크..."


### Save File

In [ ]:
import os

def save_lang_dataset(df, lang: str, save_dir: str = "data"):
    required_cols = ["text", "clean_text", "tokens", "tokens_lemmatized", "tokens_norm"]
    
    # add language label
    df_to_save = df[required_cols].copy()
    df_to_save["label"] = lang
    
    os.makedirs(save_dir, exist_ok=True)
    output_path = os.path.join(save_dir, f"output_{lang}.parquet")  
    df_to_save.to_parquet(output_path, index=False)
    display(df_to_save)


In [26]:
save_lang_dataset(df_sample_ko, lang="ko")

,text,clean_text,tokens,tokens_lemmatized,tokens_norm,label
623706,카타쿠라 카네타로 ()는 현재의 카타쿠라 공업의 대대로 이어진 이름이다\n\n 카타...,카타쿠라 카네타로 는 현재의 카타쿠라 공업의 대대로 이어진 이름이다 카타쿠라 카네타...,"[카타쿠라 카네타로, 는, 현재, 의, 카타쿠라, 공업, 의, 대대, 로, 이어지,...","[카타쿠라 카네타로, 늘, 현재, 의, 카타쿠, 공업, 의, 대대, 로, 이어, ᆫ...","[카타쿠라 카네타로, 현재, 카타쿠, 공업, 대대, 이어, 이름, 카타쿠라 카네타로...",ko
23304,"탄두(彈頭, )란 미사일, 로켓, 어뢰 등의 추진에 의해 전달되는 폭발성 물질 또는...",탄두 란 미사일 로켓 어뢰 등의 추진에 의해 전달되는 폭발성 물질 또는 독성 물질이...,"[탄두, 이, 란, 미사일, 로켓, 어뢰, 등, 의, 추진, 에, 의하, 어, 전달...","[탄두, 이, 란, 미사일, 로켓, 어뢰, 등, 의, 추진, 에, 의하, 어, 전달...","[탄두, 미사일, 로켓, 어뢰, 추진, 의하, 전달, 폭발, 물질, 또는, 물질, ...",ko
79240,"니스(, 니사르드 , , , , )는 프랑스 남부의 항만 도시로 프랑스의 지중해 연...",니스 니사르드 는 프랑스 남부의 항만 도시로 프랑스의 지중해 연안에 위치해 있다 마...,"[니스, 니사르드, 는, 프랑스, 남부, 의, 항만, 도시, 로, 프랑스, 의, 지...","[니스, 니사르드, 늘, 프랑스, 남부, 의, 항만, 도시, 로, 프랑스, 의, 지...","[니스, 니사르드, 프랑스, 남부, 항만, 도시, 프랑스, 지중해, 연안, 위치, ...",ko
140214,김홍수 게이트는 90년대 초 초등학교 동창인 법조인 친구를 통해 서울지방법원에서 근...,김홍수 게이트는 90년대 초 초등학교 동창인 법조인 친구를 통해 서울지방법원에서 근...,"[김홍, 수, 게이트, 는, 90, 년대, 초, 초등, 학교, 동창, 이, ᆫ, 법...","[김홍, 수, 게이트, 늘, 90, 년대, 초, 초등, 학교, 동창, 이, ᆫ, 법...","[김홍, 게이트, 90, 년대, 초등, 학교, 동창, 법조인, 친구, 통하, 서울,...",ko
28599,다음은 인도아리아어군의 언어 목록이다.\n\n\n 고전어\n 베다 산스크리트어\n ...,다음은 인도아리아어군의 언어 목록이다 고전어 베다 산스크리트어 산스크리트어 미탄니어...,"[다음, 은, 인도아리아어군, 의, 언어, 목록, 이, 다, 고, 전어, 베다, 산...","[다음, 은, 인도아리아어군, 의, 언어, 목록, 이, 다, 고, 전어, 베, 산스...","[인도아리아어군, 언어, 목록, 전어, 산스크리트어, 산스크리트어, 미탄니, 프라크...",ko
...,...,...,...,...,...,...
499474,《앵그리버드 트랜스포머》()는 2014년에 출시된 로비오 엔터테인먼트의 비디오 게임...,앵그리버드 트랜스포머는 2014년에 출시된 로비오 엔터테인먼트의 비디오 게임이다 평...,"[앵그리버드, 트랜스포머, 는, 2014, 년, 에, 출시, 되, ᆫ, 로비오, 엔...","[앵그리버드, 트랜스포머, 늘, 2014, 년, 에, 출시, 되, ᆫ, 로비, 엔터...","[앵그리버드, 트랜스포머, 2014, 출시, 로비, 엔터테인먼트, 비디오, 게임, ...",ko
391789,아키노 히로키(1994년 10월 8일 ~ )는 일본의 축구 선수이다. 현재 J2리그...,아키노 히로키1994년 10월 8일 는 일본의 축구 선수이다 현재 2리그의 파렌 나...,"[아키노 히로키, 1994, 년, 10, 월, 8, 일, 늘, ᆫ, 일본, 의, 축...","[아키노 히로키, 1994, 년, 10, 월, 8, 일, 늘, ᆫ, 일본, 의, 축...","[아키노 히로키, 1994, 10, 일본, 축구, 선수, 현재, 리그, 파레, 나가...",ko
176318,방길승(1971년 12월 17일 ~ )은 대한민국의 배우이다. 다른 본명은 방극현인...,방길승1971년 12월 17일 은 대한민국의 배우이다 다른 본명은 방극현인데 아내 ...,"[방길승, 1971, 년, 12, 월, 17, 일, 은, 대한민국, 의, 배우, 이...","[방길승, 1971, 년, 12, 월, 17, 일, 은, 대한민국, 의, 배우, 이...","[방길승, 1971, 12, 17, 대한민국, 배우, 본명, 아내, 리디아, 결혼,...",ko
456038,아퀼레이아의 포르투나티아누스 (Fortunatianus of Aquileia)는 4...,아퀼레이아의 포르투나티아누스 는 4세기 중엽 아퀼레이아의 주교이었다 그는 아리우스의...,"[아퀼레이아의 포르투나티아누스, 는, 4, 세기, 중엽, 아퀼레이아의, 주교, 이,...","[아퀼레이아의 포르투나티아누스, 늘, 4, 세기, 중엽, 아퀼레이아, 주교, 이, ...","[아퀼레이아의 포르투나티아누스, 세기, 중엽, 아퀼레이아, 주교, 아리우스, 가르침...",ko
